In [1]:
from lookup import LookupCreator
from sensepolar.polarity import WordPolarity
from sensepolar.embed.bertEmbed import BERTWordEmbeddings
from sensepolar.embed.gptEmbed import GPT2WordEmbeddings
from sensepolar.embed.robertaEmbed import RoBERTaWordEmbeddings
from sensepolar.polarDim import PolarDimensions
from sensepolar.oracle.dictionaryapi import Dictionary
from sensepolar.plotter import PolarityPlotter

In [2]:
def read_word_context_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    lines = content.strip().split('\n')
    words = []
    contexts = []
    for line in lines:
        split_line = [item.strip() for item in line.split(',')]
        words.append(split_line[0])
        contexts.append(split_line[1])
    return words, contexts

# read_word_context_file('data/word_context.csv')

# Bert Embeddings

In [3]:
out_path = './antonyms/'
antonym_path = "data/data_file.xlsx"

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = read_word_context_file('data/word_context.csv')
wp = WordPolarity(model, antonym_path=antonym_path, method='projection', number_polar=4)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Index: 1
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 1
Dimension: hot <------> cold
high temperature  low temperature
Value: -6.340019


Index: 4
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 2
Dimension: good <------> bad
positive negati

In [5]:
plotter = PolarityPlotter() 
plotter.plot_word_polarity(words, polar_dimensions)
plotter.plot_word_polarity_2d(words, polar_dimensions)
plotter.plot_word_polarity_polar(words, polar_dimensions)
plotter.plot_word_polarity_polar_fig(words, polar_dimensions)
plotter.plot_descriptive_antonym_pairs(words, polar_dimensions, ['rain', 'book', 'tree', 'sun'], 3)
# plotter.plot_word_polarity_2d_interactive_(words, polar_dimensions)

# GPT2 Embeddings

In [3]:
out_path = './antonyms/'
antonym_path = "data/data_file.xlsx"

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = GPT2WordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = read_word_context_file('data/word_context.csv')
wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=4)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Index: 3
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 1
Dimension: fast <------> slow
high speed  low speed
Value:9.075433731079102


Index: 1
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 2
Dimension: hot <------> cold
high temperature  l

In [4]:
plotter = PolarityPlotter() 
plotter.plot_word_polarity(words, polar_dimensions)
plotter.plot_word_polarity_2d(words, polar_dimensions)
plotter.plot_word_polarity_polar_fig(words, polar_dimensions)
plotter.plot_descriptive_antonym_pairs(words, polar_dimensions, ['rain', 'book', 'tree', 'sun'], 3)

/usr/local/anaconda3/envs/SensePOLAR/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Roberta WordEmbeddings

In [ ]:
out_path = './antonyms/'
antonym_path = "data/data_file.xlsx"

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = RoBERTaWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = read_word_context_file('data/word_context.csv')
wp = WordPolarity(model, antonym_path=antonym_path, method='projection', number_polar=4)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Index: 0
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 1
Dimension: happy <------> sad
feeling of joy feeling of sorrow
Value: -1.6392117


Index: 1
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 2
Dimension: hot <------> cold
high temperatu

In [ ]:
plotter = PolarityPlotter() 
plotter.plot_word_polarity(words, polar_dimensions)
plotter.plot_word_polarity_2d(words, polar_dimensions)
plotter.plot_word_polarity_polar_fig(words, polar_dimensions)
plotter.plot_descriptive_antonym_pairs(words, polar_dimensions, ['rain', 'book', 'tree', 'sun'], 3)

/usr/local/anaconda3/envs/SensePOLAR/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Alberta Word Embeddings

In [ ]:
out_path = './antonyms/'
antonym_path = "data_file.xlsx"

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = RoBERTaWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = read_word_context_file('data/word_context.csv')
wp = WordPolarity(model, antonym_path=antonym_path, method='projection', number_polar=4)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Index: 0
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 1
Dimension: happy <------> sad
feeling of joy feeling of sorrow
Value: -1.6392117


Index: 1
[[['happy', ['i feel happy today ']], ['sad', ['he looks sad ']]], [['hot', ['it s hot outside ']], ['cold', ['the ice is cold to touch ']]], [['big', ['the elephant is big ']], ['small', ['the ant is small ']]], [['fast', ['the cheetah runs fast ']], ['slow', ['the snail moves slow ']]], [['good', ['eating healthy food is good for you ']], ['bad', ['smoking is bad for your health ']]]]
Top: 2
Dimension: hot <------> cold
high temperatu

In [ ]:
plotter = PolarityPlotter() 
plotter.plot_word_polarity(words, polar_dimensions)
plotter.plot_word_polarity_2d(words, polar_dimensions)
plotter.plot_word_polarity_polar_fig(words, polar_dimensions)
plotter.plot_descriptive_antonym_pairs(words, polar_dimensions, ['rain', 'book', 'tree', 'sun'], 3)

/usr/local/anaconda3/envs/SensePOLAR/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Others

In [5]:
out_path = '../SensePOLAR-TeamProject/antonyms/'
# antonym_path = "data_file.xlsx"
dims = [['hot', 'cold'], ['bad', 'good'], ['intelligent', 'unintelligent'], ['capable', 'incapable']]

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
# lookupSpace = LookupCreator(dictionary, out_path, antonym_pairs=dims)
# lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = ['fire', "sun"], ["the fire is burning", "sun is shining"]
wp = WordPolarity(model, antonym_path=antonym_path,lookup_path=out_path, method='projection', number_polar=2)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading antonyms from ../SensePOLAR-TeamProject/antonyms/lookup_anto_example_dict.pkl
Index: 0
[[['test', ['test is first example ', 'second sentence example of test ']], ['test2', ['test2 is first example ', 'second sentence example of test2 ']]], [['one', ['example for one ', 'one example sentence ']], ['two', ['example for two ', 'two example sentence ']]]]
Top: 1
Dimension: test <------> test2
   
Value: -0.87755775


Index: 1
[[['test', ['test is first example ', 'second sentence example of test ']], ['test2', ['test2 is first example ', 'second sentence example of test2 ']]], [['one', ['example for one ', 'one example sentence ']], ['two', ['example for two ', 'two example sentence ']]]]
Top: 2
Dimension: one <------> two
   
Value:0.5780946016311646


Index: 0
[[['test', ['test is first example ', 'second sentence example of test ']], ['test2', ['test2 is first example ', 'second sentence example of test2 ']]], [['one', ['example for one ', 'one example sentence ']], ['two', ['e

In [19]:
plotter = PolarityPlotter() 
plotter.plot_word_polarity(words, polar_dimensions)


In [20]:
out_path = './antonyms/'
# antonym_path = "data_file.xlsx"
dims = [['hot', 'cold'], ['bad', 'good'], ['intelligent', 'unintelligent'], ['capable', 'incapable']]

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, out_path, antonym_pairs=dims)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = ['fire', "sun"], ["the fire is burning", "sun is shining"]
wp = WordPolarity(model, antonym_path=antonym_path,lookup_path=out_path, method='projection', number_polar=2)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

plotter.plot_word_polarity(words, polar_dimensions)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{"['hot', 'cold']": {'hot': ['hot stove ', 'hot water ', 'a hot august day ', 'a hot stuffy room ', 'she s hot and tired ', 'a hot forehead ', 'the fighting became hot and heavy ', 'a hot engagement ', 'a hot temper ', 'a hot topic ', 'a hot new book ', 'a hot love affair ', 'a hot argument ', 'hot pink ', 'was hot for her ', 'hot pants ', 'hot merchandise ', 'a hot car ', 'a hot sports car ', 'got off to a hot start ', 'in hot pursuit ', 'a red hot line drive ', 'a hot suspect ', 'hot salsa ', 'jalapeno peppers are very hot ', 'a hot drummer ', 'he s hot tonight ', 'one of the hot young talents ', 'cabbage patch dolls were hot last season ', 'make it hot for him ', 'in the hot seat ', 'in hot water ', 'news hot off the press ', 'red hot information ', 'hot at craps ', 'the dice are hot tonight ', 'he s hot at math but not so hot at history ', 'a hot scent ', 'hot for travel ', 'you are hot ', 'hot fuel rods ', 'a hot laboratory ', 'a hot wire ', 'a hot week on the stock market '], 'co

In [21]:
# plotter.plot_word_polarity_2d(words, polar_dimensions)

In [22]:
out_path = '../SensePOLAR-TeamProject/antonyms/'
# antonym_path = "data_file.xlsx"
dims = [['hot', 'cold'], ['bad', 'good'], ['intelligent', 'unintelligent'], ['capable', 'incapable']]

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
# lookupSpace = LookupCreator(dictionary, out_path, antonym_pairs=dims)
# lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = ['fire', "sun"], ["the fire is burning", "sun is shining"]
wp = WordPolarity(model, antonym_path=antonym_path,lookup_path=out_path, method='base-change', number_polar=2)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

plotter.plot_word_polarity(words, polar_dimensions)
plotter.plot_word_polarity_2d(words, polar_dimensions)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{"['test', 'test2']": {'test': ['test is first example ', 'second sentence example of test '], 'test2': ['test2 is first example ', 'second sentence example of test2 ']}, "['one', 'two']": {'one': ['example for one ', 'one example sentence '], 'two': ['example for two ', 'two example sentence ']}}
Loading antonyms from ../SensePOLAR-TeamProject/antonyms/lookup_anto_example_dict.pkl
Index: 0
[[['test', ['test is first example ', 'second sentence example of test ']], ['test2', ['test2 is first example ', 'second sentence example of test2 ']]], [['one', ['example for one ', 'one example sentence ']], ['two', ['example for two ', 'two example sentence ']]]]
Top: 1
Dimension: test <------> test2
   
Value: -0.0929523


Index: 1
[[['test', ['test is first example ', 'second sentence example of test ']], ['test2', ['test2 is first example ', 'second sentence example of test2 ']]], [['one', ['example for one ', 'one example sentence ']], ['two', ['example for two ', 'two example sentence ']]]]

In [35]:
def get_most_descriptive_antonym_pairs(words, polar_dimensions, inspect_words, n):
    """
    Retrieves the common antonym pairs that best describe the inspected words.

    Args:
        words (list): A list of all words.
        polar_dimensions (list): A list of polar dimensions.
        inspect_words (list): A subset of words to inspect.
        n (int): Number of antonym pairs to retrieve.

    Returns:
        list: The common antonym pairs that best describe the inspected words,
              along with the polarity values.

    Note:
        This implementation assumes that each word has only one antonym pair associated with it.
    """
    word_dict = {word: None for word in words}
    print(word_dict)
    for idx, item in enumerate(polar_dimensions):
        antonym_dict = {}
        for antonym1, antonym2, value in item:
            antonym_dict[(antonym1, antonym2)] = value
        word_dict[words[idx]] = antonym_dict

    common_antonyms = set(word_dict[inspect_words[0]].keys())
    for word in inspect_words[1:]:
        common_antonyms.intersection_update(word_dict[word].keys())

    descriptive_pairs = []
    scores = []

    for antonym in common_antonyms:
        score = sum(abs(word_dict[word][antonym]) for word in inspect_words)
        descriptive_pairs.append(antonym)
        scores.append(score)

    sorted_pairs = [pair for _, pair in sorted(zip(scores, descriptive_pairs), reverse=True)]

    if len(sorted_pairs) > n:
        sorted_pairs = sorted_pairs[:n]

    result = []
    for pair in sorted_pairs:
        polarities = [word_dict[word][pair] for word in inspect_words]
        result.append((pair, polarities))

    return result[:n]


In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as colors

def plot_descriptive_antonym_pairs(words, descriptive_pairs):
    """
    Plots each word against the descriptive antonym pairs using a horizontal bar plot.

    Args:
        words (list): A list of words.
        descriptive_pairs (list): The descriptive antonym pairs, along with the polarity values.

    Returns:
        None
    """
    fig = make_subplots(rows=len(descriptive_pairs), cols=1, shared_xaxes=True, subplot_titles=[f"{pair[0][0]}-{pair[0][1]}" for pair in descriptive_pairs])

    #take max absolute value from all polarity values
    scale = max(max([abs(n) for n in polars]) for _, polars in descriptive_pairs)
    min_polarity = -scale
    max_polarity = scale

    color_sequence = colors.qualitative.Plotly

    for i, (antonym_pair, polarity_values) in enumerate(descriptive_pairs):
        fig_idx = i + 1

        # Assign colors to words in each antonym pair
        color_map = {word: color_sequence[j % len(color_sequence)] for j, word in enumerate(words)}
        legend_names = []  # List to store legend names for each word in the antonym pair

        for j, word in enumerate(words):
            if word not in legend_names:
                legend_names.append(word)
                fig.add_trace(go.Bar(
                    y=[word],
                    x=[polarity_values[j]],
                    name=word,
                    marker=dict(color=color_map[word]),  # Assign the color for each word
                    orientation='h',
                    showlegend=False,
                    offsetgroup=f"Pair {fig_idx}"
                ), row=fig_idx, col=1)

    fig.update_layout(
        title="Word Polarity for Descriptive Antonym Pairs",
        xaxis_title="Polarity",
        barmode="group",
        legend_traceorder="reversed",
        xaxis=dict(range=[min_polarity, max_polarity])  # Set the x-axis range from min to max polarity values
    )

    fig.update_yaxes(showticklabels=False)

    fig.show()


In [34]:
get_most_descriptive_antonym_pairs(words, polar_dimensions, ['tree', 'rain'], 4)

{'fire': None, 'sun': None, 'tree': None, 'rain': None}


AttributeError: 'NoneType' object has no attribute 'keys'

In [27]:
plot_descriptive_antonym_pairs(['sun', 'rain'], get_most_descriptive_antonym_pairs(words, polar_dimensions, ['tree', 'rain'], 4))

KeyError: 'tree'

In [ ]:
get_most_descriptive_antonym_pairs(words, polar_dimensions, ['tree', 'rain'], 4)

[(('big', 'small'), [-1.1064599, -2.539312]),
 (('good', 'bad'), [-0.7773825, 0.6468782]),
 (('hot', 'cold'), [-0.35254246, -1.0438247])]